This code is designed to merge the credits dataframe and the tmdb details dataframe into a dataframe called merged_movie_details_credits.csv

In [1]:
!pip install tmdbv3api

In [1]:
from tmdbv3api import TMDb
from tmdbv3api import Movie
from tmdbv3api.exceptions import TMDbException
import random
import pandas as pd
import csv
import numpy as np
import ast
from math import exp
from datetime import datetime
tmdb=TMDb()
tmdb.api_key='9cf68f4c97c8f0cc6bb9646da389a808'

>The next cell will fetch the actor's star score in the year of the movie's release by their id and apply the predefined weighting function to modify the star score based on the billing order.

In [4]:
# Initialize csv file paths
csv_file_path1='people_info_star_scores_merged.csv' 
csv_file_path2='merged_data_mod_scores_filtered_US_release_year.csv'
# Create DataFrames from both CSV files
df1 = pd.read_csv(csv_file_path1)
df2 = pd.read_csv(csv_file_path2)

In [9]:
star_scores = []  # Create a list to collect star scores
for index, row in df2.iterrows():
    if(row['Release Year'] > 2012):
        row_cast_info = row['cast_ids']  # Extract cast_ids array
        if not pd.isna(row_cast_info):  # Check NaN
            row_cast_info = ast.literal_eval(row_cast_info)
        else:
            row_cast_info = []  # Set to an empty list if NaN
        star_scores_row = []
        for cast_id in row_cast_info:
            release_year = row['Release Year']
            previous_year = release_year - 1
            star_column_name = f'star_{previous_year}'  # Create the corresponding column name
            cast_info_row = df1[df1['id'] == cast_id]  # Get the row containing queried cast id
            if not cast_info_row.empty:  # Check if cast id in people_info_star_scores is not empty
                star_score = cast_info_row[star_column_name].values[0]
                star_scores_row.append(star_score)
        star_scores.append(star_scores_row)  # Append that row array to a singular entry in the corresponding row in df2
    else:
        star_scores.append([])

# Create an output dataframe
df_output = df2
df_output['Raw Star Scores'] = star_scores

# Save to csv
df_output.to_csv('merged_data_mod_scores_filtered_US_release_year_raw_scores.csv', index=False)
